# Spider Datagen

Create Spark SQL eventlogs from the [Spider Text-to-SQL](https://yale-lily.github.io/spider) dataset, which contains pairs of natural language queries and SQL equivalents.  The dataset also includes [sqlite databases](https://drive.google.com/u/0/uc?id=1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m&export=download) used to evaluate the SQL queries.

### Using sqlite3

Example of using sqlite3 python API to access one of the Spider databases.

In [ ]:
import sqlite3

In [ ]:
database = '/home/leey/devpub/spider/spider/database/college_1/college_1.sqlite'

In [ ]:
con = sqlite3.connect(database)
cur = con.cursor()

In [ ]:
# Example query from dataset
res = cur.execute("""
SELECT T1.stu_fname 
FROM student AS T1 
JOIN enroll AS T2 ON T1.stu_num  =  T2.stu_num 
JOIN CLASS AS T3 ON T2.class_code  =  T3.class_code 
JOIN course AS T4 ON T3.crs_code  =  T4.crs_code 
JOIN department AS T5 ON T5.dept_code  =  T4.dept_code 
WHERE T5.dept_name  =  'Accounting' 
INTERSECT 
SELECT T1.stu_fname FROM student AS T1 
JOIN enroll AS T2 ON T1.stu_num  =  T2.stu_num 
JOIN CLASS AS T3 ON T2.class_code  =  T3.class_code 
JOIN course AS T4 ON T3.crs_code  =  T4.crs_code 
JOIN department AS T5 ON T5.dept_code  =  T4.dept_code 
WHERE T5.dept_name  =  'Computer Info. Systems'
""")

In [ ]:
res.fetchall()

### Importing sqlite DB into pyspark
Per https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/from_to_dbms.html

In [ ]:
import pandas as pd
import pyspark.pandas as ps
import pyspark.sql.functions as F
import sqlite3

from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType

In [ ]:
database = '/home/leey/devpub/spider/spider/database/college_1/college_1.sqlite'

In [ ]:
con = sqlite3.connect(database)
cur = con.cursor()

In [ ]:
res = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = res.fetchall()
tables = [x[0] for x in tables]
tables

In [ ]:
# Load each table into spark
for table in tables:
    print(table)
    df = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable=table, \
                 url='jdbc:sqlite:/home/leey/devpub/spider/spider/database/college_1/college_1.sqlite') \
        .option("customSchema", "STU_DOB STRING, EMP_HIREDATE STRING, EMP_DOB STRING") \
        .load()
    df.show()
    df.createOrReplaceTempView(table)

### Running Spider queries in Spark SQL

In [ ]:
from datasets import load_dataset

dataset_name = "xlangai/spider"
dataset = load_dataset(dataset_name, split="train")
df = pd.DataFrame(dataset)
df

In [ ]:
queries = list(df['query'].loc[df['db_id'] == 'college_1'])

In [ ]:
%%time
# run all queries for the 'college_1' database
for i, q in enumerate(queries):
    try:
        print(f"{i}: {q}")
        res = spark.sql(q)
        res.show()
    except Exception as e:
        print(e)

### Generate datagen.py

Use the [Databricks Labs Data Generator](https://github.com/databrickslabs/dbldatagen) to increase the size of the dataset.

Note that the generator prints out pyspark/python code, so we write the output as a file to execute via Spark per instructions below.

In [ ]:
import dbldatagen as dg

In [ ]:
with open("datagen.py", "w") as f:
    f.write(
"""
import os
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("spider-datagen")
    .config("spark.jars", "/home/leey/devpub/spider/sqlite-jdbc-3.45.2.0.jar")
    .config("spark.driver.extraClassPath", "/home/leey/devpub/spider/sqlite-jdbc-3.45.2.0.jar")
    .getOrCreate()
)

"""
    )
    for table in tables:
        df = spark.table(table)
        # df.show()
        analyzer = dg.DataAnalyzer(sparkSession=spark, df=df)
        generatedCode = analyzer.scriptDataGeneratorFromData()
        f.write(f"# {table}\n")
        f.writelines(generatedCode)
        f.write("\n")
        f.write("df_synthetic = generation_spec.build()\n")
        f.write(f"df_synthetic.write.mode('overwrite').parquet('{table}.parquet')\n")
        f.write("\n")

### Run datagen.py on command line

```bash
spark-submit --master $MASTER datagen.py
```

### Merge synthetic w/ "real" data

The synthetic data does not contain any values included in the Spider SQL queries, e.g. `WHERE T5.dept_name  =  'Computer Info. Systems'`, so we need to merge the synthetic data with the original data in order to satisfy the SQL queries.

In [ ]:
for table in tables:
    df = spark.table(table)
    synthetic_df = spark.read.parquet(f"{table}.parquet")
    merged_df = synthetic_df.union(df)
    # merged_df.cache()
    merged_df.createOrReplaceTempView(table)
    merged_df.write.mode("overwrite").parquet(f"syn_{table}.parquet")
    print(merged_df.count())

### Re-running Spider queries in Spark SQL

Now, we can run the Spider SQL queries against the scaled up database (excluding certain slow queries for now).

In [ ]:
from datetime import datetime

In [ ]:
%%time
slow_queries = [34, 35, 78, 79, 120, 121, 122, 127, 140, 141, 142, 143, 144, 145, 148, 149, 150, 151]
for i, q in enumerate(queries):
    if i not in slow_queries:
        continue
    try:
        print(f"{datetime.utcnow()}: {i}: {q}")
        res = spark.sql(q)
        res.show()
    except Exception as e:
        print(e)

In [ ]:
q = """
SELECT count(*) FROM CLASS AS T1
JOIN enroll AS T2 ON T1.class_code  =  T2.class_code
JOIN course AS T3 ON T1.crs_code  =  T3.crs_code
JOIN department AS T4 ON T3.dept_code  =  T4.dept_code
WHERE T4.dept_name  =  'Accounting'
""".strip()

In [ ]:
%%time
res = spark.sql(q)
res.show()

### Scratch

In [1]:
%%time
import os
import pandas as pd
import sqlite3
from datasets import load_dataset
from datetime import datetime
from pyspark.sql import SparkSession

database = '/home/leey/devpub/spider/spider/database/college_1/college_1.sqlite'

con = sqlite3.connect(database)
cur = con.cursor()
res = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = res.fetchall()
tables = [x[0] for x in tables]
print(tables)
con.close()

dataset_name = "xlangai/spider"
dataset = load_dataset(dataset_name, split="train")
df = pd.DataFrame(dataset)
queries = list(df['query'].loc[df['db_id'] == 'college_1'])

for table in tables:
    df = spark.read.parquet(f"/home/leey/devpub/spider/notebooks/syn_{table}.parquet")
    df.createOrReplaceTempView(table)
    print(df.count())

slow_queries = [34, 35, 78, 79, 120, 121, 122, 127, 140, 141, 142, 143, 144, 145, 148, 149, 150, 151]
for i, q in enumerate(queries):
    if i not in slow_queries:
        continue
    try:
        print(f"{datetime.utcnow()}: {i}: {q}")
        res = spark.sql(q)
        res.show()
    except Exception as e:
        print(e)


['CLASS', 'COURSE', 'DEPARTMENT', 'EMPLOYEE', 'ENROLL', 'PROFESSOR', 'STUDENT']


10013
10006
10011
10037
10006
10022
10008
2024-04-18 00:22:35.809490: 34: SELECT count(*) ,  T3.school_code FROM CLASS AS T1 JOIN course AS T2 ON T1.crs_code  =  T2.crs_code JOIN department AS T3 ON T2.dept_code  =  T3.dept_code GROUP BY T3.school_code


+--------+-----------+
|count(1)|school_code|
+--------+-----------+
| 6498144|     veniam|
| 3856320|     cillum|
| 2477088|      velit|
| 6067680|      lorem|
| 4480416|    laborum|
| 4726368|     fugiat|
| 4849344|        non|
| 5036544|    eiusmod|
| 7556928|         ex|
| 6709632|    nostrud|
| 2791584|        sit|
| 5156352|    laboris|
| 8028192|consectetur|
| 4533696|     aliqua|
| 2206656|   occaecat|
|12364416|         in|
| 6493536|       aute|
| 4709088|       enim|
| 6356096|    commodo|
| 4865568|       sunt|
+--------+-----------+
only showing top 20 rows

2024-04-18 00:22:42.254997: 35: SELECT count(*) ,  T3.school_code FROM CLASS AS T1 JOIN course AS T2 ON T1.crs_code  =  T2.crs_code JOIN department AS T3 ON T2.dept_code  =  T3.dept_code GROUP BY T3.school_code


+--------+-----------+
|count(1)|school_code|
+--------+-----------+
| 6498144|     veniam|
| 3856320|     cillum|
| 2477088|      velit|
| 6067680|      lorem|
| 4480416|    laborum|
| 4726368|     fugiat|
| 4849344|        non|
| 5036544|    eiusmod|
| 7556928|         ex|
| 6709632|    nostrud|
| 2791584|        sit|
| 5156352|    laboris|
| 8028192|consectetur|
| 4533696|     aliqua|
| 2206656|   occaecat|
|12364416|         in|
| 6493536|       aute|
| 4709088|       enim|
| 6356096|    commodo|
| 4865568|       sunt|
+--------+-----------+
only showing top 20 rows

2024-04-18 00:22:46.368742: 78: SELECT count(*) FROM CLASS AS T1 JOIN enroll AS T2 ON T1.class_code  =  T2.class_code JOIN course AS T3 ON T1.crs_code  =  T3.crs_code JOIN department AS T4 ON T3.dept_code  =  T4.dept_code WHERE T4.dept_name  =  'Accounting'


+--------+
|count(1)|
+--------+
|       2|
+--------+

2024-04-18 00:22:58.690247: 79: SELECT count(*) FROM CLASS AS T1 JOIN enroll AS T2 ON T1.class_code  =  T2.class_code JOIN course AS T3 ON T1.crs_code  =  T3.crs_code JOIN department AS T4 ON T3.dept_code  =  T4.dept_code WHERE T4.dept_name  =  'Accounting'


...

+---------+
|stu_fname|
+---------+
|   aliqua|
|     sunt|
|     Anne|
|       ad|
|  William|
+---------+

2024-04-18 00:25:06.429179: 122: SELECT avg(T2.stu_gpa) FROM enroll AS T1 JOIN student AS T2 ON T1.stu_num  =  T2.stu_num JOIN CLASS AS T3 ON T1.class_code  =  T3.class_code WHERE T3.crs_code  =  'ACCT-211'
+----------------+
|    avg(stu_gpa)|
+----------------+
|2.90799994468689|
+----------------+

2024-04-18 00:25:06.684126: 127: SELECT T2.dept_name FROM student AS T1 JOIN department AS T2 ON T1.dept_code  =  T2.dept_code ORDER BY stu_gpa LIMIT 1
+---------+
|dept_name|
+---------+
|     enim|
+---------+

2024-04-18 00:25:07.166831: 140: SELECT T2.emp_fname ,  T4.prof_office ,  T3.crs_description FROM CLASS AS T1 JOIN employee AS T2 ON T1.prof_num  =  T2.emp_num JOIN course AS T3 ON T1.crs_code  =  T3.crs_code JOIN professor AS T4 ON T2.emp_num  =  T4.emp_num
+---------+-------------+---------------+
|emp_fname|  prof_office|crs_description|
+---------+-------------+-------

+---------+-----------+---------------+---------+
|emp_fname|prof_office|crs_description|dept_name|
+---------+-----------+---------------+---------+
|   veniam|         ex|        officia|       in|
|   veniam|         ex|        officia|    ipsum|
|   veniam|         ex|        officia|      sit|
|   veniam|         ex|        officia|     quis|
|   veniam|         ex|        officia|     sint|
|   veniam|         ex|        officia|      non|
|   veniam|         ex|        officia|      qui|
|   veniam|         ex|        officia|      non|
|   veniam|         ex|        officia|     esse|
|   veniam|         ex|        officia|     amet|
|   veniam|         ex|        officia|       id|
|   veniam|         ex|        officia|       ea|
|   veniam|         ex|        officia|       ex|
|   veniam|         ex|        officia|     sunt|
|   veniam|         ex|        officia|     aute|
|   veniam|         ex|        officia|       ex|
|   veniam|         ex|        officia|       eu|


+---------+-----------+---------------+---------+
|emp_fname|prof_office|crs_description|dept_name|
+---------+-----------+---------------+---------+
|   veniam|         ex|        officia|       in|
|   veniam|         ex|        officia|    ipsum|
|   veniam|         ex|        officia|      sit|
|   veniam|         ex|        officia|     quis|
|   veniam|         ex|        officia|     sint|
|   veniam|         ex|        officia|      non|
|   veniam|         ex|        officia|      qui|
|   veniam|         ex|        officia|      non|
|   veniam|         ex|        officia|     esse|
|   veniam|         ex|        officia|     amet|
|   veniam|         ex|        officia|       id|
|   veniam|         ex|        officia|       ea|
|   veniam|         ex|        officia|       ex|
|   veniam|         ex|        officia|     sunt|
|   veniam|         ex|        officia|     aute|
|   veniam|         ex|        officia|       ex|
|   veniam|         ex|        officia|       eu|


+---------+---------+---------------+
|stu_fname|stu_lname|crs_description|
+---------+---------+---------------+
|  aliquip|  commodo|     adipiscing|
|  aliquip|  commodo|          minim|
|  aliquip|  commodo|             ut|
|  aliquip|  commodo|           elit|
|  aliquip|  commodo|             ut|
|  aliquip|  commodo|         labore|
|  aliquip|  commodo|        officia|
|  aliquip|  commodo|           elit|
|  aliquip|  commodo|       deserunt|
|  aliquip|  commodo|             do|
|  aliquip|  commodo|         cillum|
|  aliquip|  commodo|          irure|
|  aliquip|  commodo|          irure|
|  aliquip|  commodo|        laboris|
|  aliquip|  commodo|         dolore|
|  aliquip|  commodo|     adipiscing|
|  aliquip|  commodo|          minim|
|  aliquip|  commodo|             ut|
|  aliquip|  commodo|           elit|
|  aliquip|  commodo|             ut|
+---------+---------+---------------+
only showing top 20 rows

2024-04-18 00:25:51.429976: 145: SELECT T1.stu_fname ,  T1.stu

+-------------+----------------+
|    emp_fname|prof_high_degree|
+-------------+----------------+
|    consequat|           Ph.D.|
|     proident|           Ph.D.|
|   adipiscing|           Ph.D.|
|           ut|           Ph.D.|
|      nostrud|           Ph.D.|
|reprehenderit|           Ph.D.|
|         quis|           Ph.D.|
|           in|           Ph.D.|
|           et|           Ph.D.|
|      eiusmod|           Ph.D.|
|      laborum|           Ph.D.|
|       labore|           Ph.D.|
|     occaecat|           Ph.D.|
|         sunt|           Ph.D.|
|         elit|           Ph.D.|
|           ea|           Ph.D.|
|          sit|           Ph.D.|
|        dolor|           Ph.D.|
|          sed|           Ph.D.|
|   incididunt|           Ph.D.|
+-------------+----------------+
only showing top 20 rows

2024-04-18 00:25:53.062288: 151: SELECT DISTINCT T2.emp_fname ,  T3.prof_high_degree FROM CLASS AS T1 JOIN employee AS T2 ON T1.prof_num  =  T2.emp_num JOIN professor AS T3 ON T2.emp